## dataset functions

In [1]:
# dataset = "exp_density"
# from exp_density_polynomial import *
dataset = "cvar"
from cvar import *
# dataset = "energy"
# from energy import *
# dataset = "kelly"
# from kelly import *
# dataset = "news_vendor"
# from news_vendor import *

print(cp.__version__)

device = cpu
1.1.12


## osmm

In [2]:
from osmm import OSMM
osmm_prob = OSMM(f_torch=my_f_torch, g_cvxpy=my_g_cvxpy)

## generate random data

In [3]:
W = generate_random_data()
W_validation = generate_random_data()
init_val = get_initial_val()

## solve

In [4]:
result = osmm_prob.solve(W, init_val, W_validate=W_validation, 
                         solver="MOSEK", verbose=True)

iter = 10, objf = -1.347e+00, lower bound = -1.354e+00, RMS residual = 7.232e-02, sampling acc = 3.241e-03, ||G||_F = 1.045e+02
iter = 20, objf = -1.348e+00, lower bound = -1.348e+00, RMS residual = 5.474e-03, sampling acc = 1.930e-03, ||G||_F = 3.278e+02
      Terminated. Num iterations = 20, objf = -1.348e+00, lower bound = -1.348e+00, RMS residual = 5.474e-03, sampling acc = 1.930e-03.
      Time elapsed (secs): 55.816046.



In [5]:
print("result =", result)

result = -1.3476430177688599


## get results

In [6]:
x_soln = osmm_prob.method_results["soln"]
Xs = osmm_prob.method_results["var_iters"]

In [7]:
max_num_rounds = len(osmm_prob.method_results["objf_iters"])
iters_taken = osmm_prob.method_results["total_iters"]

In [8]:
objfs = np.array(osmm_prob.method_results["objf_iters"])
for i in range(1, iters_taken):
    objfs[i] = np.min(osmm_prob.method_results["objf_iters"][0:i + 1])
objfs[iters_taken::] = objfs[iters_taken]
objf_min = np.min(objfs[0:iters_taken + 1])

In [9]:
objfs_val = np.array(osmm_prob.method_results["objf_validate_iters"])
objfs_val[iters_taken::] = objfs_val[iters_taken]

In [10]:
lower_bounds = np.array(osmm_prob.method_results["lower_bound_iters"])
lower_bounds[iters_taken::] = lower_bounds[iters_taken]

In [11]:
rms_res_norms = np.array(osmm_prob.method_results["rms_res_iters"])
start_idx = 1
for i in range(start_idx, iters_taken):
    rms_res_norms[i] = np.min(osmm_prob.method_results["rms_res_iters"][start_idx:i + 1])
rms_res_norms[iters_taken::] = rms_res_norms[iters_taken]

In [12]:
time_detail = osmm_prob.method_results["time_detail_iters"]
print(time_detail.shape)
print(time_detail[2, 0:osmm_prob.method_results["total_iters"] + 1])
print(time_detail[3, 0:osmm_prob.method_results["total_iters"] + 1])
print(np.sum(time_detail[2, 0:osmm_prob.method_results["total_iters"]]) / 
      osmm_prob.method_results["total_iters"])
print(osmm_prob.method_results["total_iters"])

(4, 200)
[ 0.         39.51200795  0.07241178  0.07507133  0.0755589   0.07733893
  0.09506679  0.08551884  0.08934522  0.08703995  0.09104896  0.10282183
  0.09898901  0.10220981  0.10636902  0.12037921  0.13275599  0.11741209
  0.11723208  0.11648583  0.12445498]
[ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.         14.10273004  0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.07025695]
2.0637531757354735
20


## plots

In [ ]:
if dataset == "exp_density":
    my_plot_exp_density_one_result(Xs, objfs, iters_taken)
else:
    my_plot_one_result(W, x_soln, is_save_fig=False, figname="cvar_pnt9_result.pdf")

In [ ]:
linewidth = 2
fontsize = 16
font = {'family': 'serif',
        'size': fontsize,
        }

In [ ]:
fig = plt.figure(tight_layout=False, figsize=(16, 5))
gs = gridspec.GridSpec(1, 2)
plt.rcParams.update({'font.size': fontsize})

plot1 = fig.add_subplot(gs[0, 0])
rounds = range(0, max_num_rounds)
if osmm_prob.method_results["total_iters"] < max_num_rounds:
    plot1.axvline(osmm_prob.method_results["total_iters"], color='gray', linestyle='dashed', linewidth=linewidth)
##objf
plot1.plot(np.array(rounds), objfs, linewidth=linewidth, label="Objective value")
##objf_validation
plot1.plot(np.array(rounds), objfs_val, ':', linewidth=linewidth, label="Validation objective value")
##l_k
plot1.plot(np.array(rounds), lower_bounds, '--', linewidth=linewidth, label="Lower bound")

plot1.set_xlabel("Iterations", fontdict=font)
plot1.legend(prop=font)

#=========================================================================================================
plot2 = fig.add_subplot(gs[0, 1])
##subopt
plot2.plot(np.array(rounds), objfs - objf_min, linewidth=linewidth, label="Suboptimality")
## gaps
gaps = objfs - lower_bounds
plot2.plot(rounds, gaps, linewidth=linewidth, label="Gap")
## residual norm
plot2.plot(rounds[start_idx::], rms_res_norms[start_idx::], label="RMS residual", color='#2ca02c')

plot2.set_xlabel("Iterations", fontdict=font)
plot2.set_yscale("log")
plot2.legend(prop=font)

plot_iter_lim = iters_taken
ylim = None
if plot_iter_lim is not None:
    plot1.set_xlim([0, plot_iter_lim])
    plot2.set_xlim([0, plot_iter_lim])
if ylim is not None:
    plot2.set_ylim([ylim[0], ylim[1]])